In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import umap.umap_ as umap
# os.environ["CUDA_VISIBLE_DEVICES"]="2"
import numpy as np
import pandas as pd
import umap.umap_ as umap

from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,precision_score,recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from scipy.spatial.distance import hamming

# ASTRA
from astra.torch.data import load_cifar_10
from astra.torch.utils import train_fn
from astra.torch.models import EfficientNet, MLP, MLPClassifier, EfficientNetClassifier

from itertools import permutations,product

# %%
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/rishabh.mondal/ASTRA/astra/torch/data.py:12: UserWarning: TORCH_HOME not set, setting it to /home/rishabh.mondal/.cache/torch
  warnings.warn(f"TORCH_HOME not set, setting it to {os.environ['TORCH_HOME']}")


device(type='cuda')

In [3]:
# %%
loaded_data = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/new_bangladesh_100.pt")
X_train=loaded_data['X_train']
X_test=loaded_data['X_test']
y_train=loaded_data['y_train']
y_test=loaded_data['y_test']

In [4]:
ecf = EfficientNetClassifier(dense_hidden_dims=[1280,1024,256],n_classes=2)


In [5]:
result_dict = {}

def get_metrics(y_pred,y_label):
    with torch.no_grad():
        acc = accuracy_score(y_label,y_pred)
        f1 = f1_score(y_label,y_pred)
        precision = precision_score(y_label,y_pred)
        recall = recall_score(y_label,y_pred)
        return acc,f1,precision,recall

def predict(model,X_test,y_test,percent,ssl,batch_size = 64):
    with torch.no_grad():
        model.eval()
        # incorporating batch size
        test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        y_pred = []
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            y_pred.append(output)
        y_pred = torch.cat(y_pred)
        y_pred = torch.argmax(y_pred,dim=1)
        y_pred = y_pred.cpu().numpy()
        y_test = y_test.cpu().numpy()
        acc,f1,precision,recall = get_metrics(y_pred,y_test)


        result_dict[percent+" "+ssl] = {"accuracy":acc,"f1":f1,"precision":precision,"recall":recall}

In [6]:
path="/home/vannsh.jani/brick_kilns/githubrepo/ML/model_59_delban1.pth"
effictnet=torchvision.models.efficientnet_b0(pretrained=True)
effictnet.classifier[1]=nn.Linear(1280,2)
effictnet.features.load_state_dict(torch.load(path))
model = effictnet.to(device)


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
y_train = y_train.type(torch.uint8)


In [8]:
iter_losses, epoch_losses = train_fn(model,nn.CrossEntropyLoss(), X_train, y_train, lr=3e-4,verbose=True,batch_size=128,epochs=30)


Loss: 0.00068290: 100%|██████████| 30/30 [04:19<00:00,  8.66s/it]


In [11]:
predict(model,X_test,y_test,"100","SSL")

/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
result_dict

{'100 SSL': {'accuracy': 0.8787142857142857,
  'f1': 0.0,
  'precision': 0.0,
  'recall': 0.0}}

: 